In [1]:
import numpy as np
import pandas as pd

# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)\n",

from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [2]:
#creating geolocations for each hotel using Geopy
# source https://towardsdatascience.com/geocode-with-python-161ec1e62b89

# instatiate the Nominatim from Geopy
locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


###Ultimately not used below, once it was determine the error was unresolvable

# # 2- - create location column
# data['location'] = data['address'].apply(geocode)
# # 3 - create longitude, laatitude and altitude from location column (returns tuple)
# data['point'] = data['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# # 4 - split point column into latitude, longitude and altitude columns
# data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [3]:
# applying the geocode function above ran into errors that it could not resolve. 
# created the  function below to address this issue (no pun intended)

def get_geocodes(data):
    locationlist = []
    counter = 0
    for index, row in data.iterrows():
        #empty dictionary to hold the values of the lat and long
        geocodeadd = {}
        # this will gather geocode for specific hotel street address
        try:
            geocode = RateLimiter(locator.geocode, min_delay_seconds=5)
            location = locator.geocode(row['address'])  
#             locationlist.append(location)
            geocodeadd['name'] = row['name']
            geocodeadd['address'] = row['address']
            geocodeadd['latitude'] = location.latitude
            geocodeadd['longitude'] = location.longitude
            print(f"{counter}: Latitude = {location.latitude}, Longitude = {location.longitude}")
        # if the address of the hotel does not produce geocode the below will provide geocode for the zipcode
        except: 
            try:
                geocode = RateLimiter(locator.geocode, min_delay_seconds=5)
                location = locator.geocode(row['address2'])                 
            
                geocodeadd['name'] = row['name']
                geocodeadd['address'] = row['address']
                geocodeadd['latitude'] = location.latitude
                geocodeadd['longitude'] = location.longitude
                print(f"{counter}: Latitude = {location.latitude}, Longitude = {location.longitude}")
            # catch all for non producable geocodes
            except: 
                geocodeadd['name'] = row['name']
                geocodeadd['address'] = row['address']
                geocodeadd['latitude'] = 'N/A'
                geocodeadd['longitude'] = 'N/A'
                print(f"{counter}: Latitude = N/A, Longitude = N/A")
        locationlist.append(geocodeadd)
#         time.sleep(1)
        counter += 1
    df = pd.DataFrame(locationlist)
    final_df = pd.merge(data, df, on='name')
    return final_df

In [11]:
data = pd.read_csv('./Data/all_hotels_cleaned.csv')

In [12]:
# creating a full address column to use for getting geolocations
addresses = []
for index, row in data.iterrows():
    street = row['street']
    city = row['city']
    state = row['state']
    postal = row['postal']
    address = street +', ' + city + ', ' + state+ ' ' + postal
    addresses.append(address)
data['address'] = addresses

In [13]:
addresses2 = []
for index, row in data.iterrows():
    city = row['city']
    state = row['state']
    postal = row['postal']
    address = city + ', ' + state+ ' ' + postal
    addresses2.append(address)
data['address2'] = addresses2

In [14]:

testdata = data

In [15]:
testdata.head(2)

,Unnamed: 0,rewards,brand,name,rating,description,street,city,state,postal,url,class_id,category,address,address2
0,0,Hilton,Hampton Inn,Hampton Inn Alexander City,4.0,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,"1551 Elkahatchee Road, Alexander City, AL 35010","Alexander City, AL 35010"
1,1,Hilton,Hampton Inn,Hampton Inn Wetumpka,5.0,"We’re on the banks of the Coosa River, a short...",350 South Main Street,Wetumpka,AL,36092,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,"350 South Main Street, Wetumpka, AL 36092","Wetumpka, AL 36092"
2,2,Hilton,Hampton Inn,Hampton Inn Auburn,4.0,"We're off I-85, under 10 minutes from Chewacla...",2430 S. College St.,Auburn,AL,36832,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,"2430 S. College St., Auburn, AL 36832","Auburn, AL 36832"
3,3,Hilton,Tru,Tru by Hilton Auburn,4.0,"Make this Tru by Hilton your own. We’ll help, ...",2411 W Pace Blvd,Auburn,AL,36830,https://tru3.hilton.com/en/hotels/alabama/tru-...,5,Budget,"2411 W Pace Blvd, Auburn, AL 36830","Auburn, AL 36830"
4,4,Hilton,Hilton Garden Inn,Hilton Garden Inn Auburn/Opelika,4.0,"Our hotel is just off I-85, minutes away from ...",2555 Hilton Garden Drive,Auburn,AL,36830,https://hiltongardeninn3.hilton.com/en/hotels/...,3,Full-Service Entry,"2555 Hilton Garden Drive, Auburn, AL 36830","Auburn, AL 36830"


In [16]:
testdata = get_geocodes(testdata)

0: Latitude = 32.925230083822264, Longitude = -85.96734888230006
1: Latitude = 32.535016163265304, Longitude = -86.20561436734694
2: Latitude = 32.5781087, Longitude = -85.4975499
3: Latitude = 32.5558732, Longitude = -85.50472840014689
4: Latitude = 32.6054196, Longitude = -85.4311496
5: Latitude = 32.6137233, Longitude = -85.4017041
6: Latitude = 32.6137233, Longitude = -85.4017041
7: Latitude = 32.460234, Longitude = -86.390239
8: Latitude = 32.458578, Longitude = -86.4006169
9: Latitude = 34.7809494, Longitude = -86.9370974
10: Latitude = 34.62163121428571, Longitude = -86.99196778571428
11: Latitude = 34.607383, Longitude = -86.979821
12: Latitude = 34.671277236363636, Longitude = -86.76470254545454
13: Latitude = 34.671910999999994, Longitude = -86.7504348231146
14: Latitude = 34.729847, Longitude = -86.5859011
15: Latitude = 34.729847, Longitude = -86.5859011
16: Latitude = 34.594974, Longitude = -87.031232
17: Latitude = 34.729847, Longitude = -86.5859011
18: Latitude = 34.7298

147: Latitude = 33.482085433078915, Longitude = -112.07565187541333
148: Latitude = 33.4807791, Longitude = -112.07216184453115
149: Latitude = 33.4925805, Longitude = -112.0751922
150: Latitude = 33.5791462, Longitude = -112.11971973228715
151: Latitude = 33.58004927272727, Longitude = -112.11414281818182
152: Latitude = 33.301472149999995, Longitude = -111.84248914047943
153: Latitude = 33.2886395, Longitude = -111.88594689594271
154: Latitude = 33.2986612, Longitude = -111.8840686
155: Latitude = 33.28794499999999, Longitude = -111.8868651199042
156: Latitude = 33.262538449999994, Longitude = -111.88388090696654
157: Latitude = 33.36088548756907, Longitude = -111.89475831974578
158: Latitude = 33.306963333333336, Longitude = -111.968963
159: Latitude = 33.306964949494954, Longitude = -111.96939754545454
160: Latitude = 33.304293200000004, Longitude = -111.9702630091837
161: Latitude = 35.1918848, Longitude = -111.65889644392051
162: Latitude = 35.2190402, Longitude = -111.5912394
16

287: Latitude = 34.1508718, Longitude = -118.4489865
288: Latitude = 34.0148344, Longitude = -118.4910646
289: Latitude = 34.0250724, Longitude = -118.4965129
290: Latitude = 34.0250724, Longitude = -118.4965129
291: Latitude = 34.3623483, Longitude = -118.5552078
292: Latitude = 33.9842724, Longitude = -118.4559382
293: Latitude = 37.78803775, Longitude = -122.25334553034567
294: Latitude = 37.7245422, Longitude = -122.2424640660786
295: Latitude = 37.7245651, Longitude = -122.242373
296: Latitude = 37.8017492, Longitude = -122.27055306995928
297: Latitude = 37.838189850000006, Longitude = -122.29867496081064
298: Latitude = 37.6858759, Longitude = -122.13267964987938
299: Latitude = 37.781672571428565, Longitude = -122.40796742857142
300: Latitude = 37.79514395, Longitude = -122.4043060324974
301: Latitude = 37.6733489, Longitude = -122.3873734
302: Latitude = 37.6894464, Longitude = -122.3916056
303: Latitude = 37.6385096, Longitude = -122.0566207
304: Latitude = 37.661522262488546,

426: Latitude = 34.22105813700609, Longitude = -119.14477856037466
427: Latitude = 34.181362, Longitude = -119.2256501
428: Latitude = 34.3908191, Longitude = -118.5736432
429: Latitude = 34.4440119, Longitude = -118.6012775
430: Latitude = 37.28701840816326, Longitude = -121.931647
431: Latitude = 37.3292414, Longitude = -121.89046715432025
432: Latitude = 37.301495201527636, Longitude = -121.85728528321293
433: Latitude = 37.3228934, Longitude = -122.0322895
434: Latitude = 37.321637949999996, Longitude = -122.0313007258109
435: Latitude = 37.362694, Longitude = -121.908033
436: Latitude = 37.370999035159606, Longitude = -121.92328171948594
437: Latitude = 37.37508352510985, Longitude = -121.91984211611066
438: Latitude = 37.384559949999996, Longitude = -121.92828765598873
439: Latitude = 37.2593775, Longitude = -121.8194551
440: Latitude = 37.38390527011897, Longitude = -121.97937192094477
441: Latitude = 37.395117132398454, Longitude = -122.02903627026521
442: Latitude = 37.3987806

564: Latitude = 37.5219812, Longitude = -121.99001765364781
565: Latitude = 37.5232841, Longitude = -122.00244048172787
566: Latitude = 37.49463717515967, Longitude = -121.9326460916129
567: Latitude = 37.5982445, Longitude = -122.06832085080214
568: Latitude = 37.4346113, Longitude = -121.89190055282646
569: Latitude = 36.99108021620853, Longitude = -121.56180682581062
570: Latitude = 36.986376122270755, Longitude = -121.55689399335272
571: Latitude = 37.1210556, Longitude = -121.6244540472
572: Latitude = 36.903723270314124, Longitude = -121.78234998794478
573: Latitude = 36.67073639759036, Longitude = -121.64212282931726
574: Latitude = 36.98505964130531, Longitude = -122.02368492391594
575: Latitude = 37.033458, Longitude = -122.022965
576: Latitude = 36.9713156, Longitude = -122.036188
577: Latitude = 36.62593, Longitude = -121.8169383456945
578: Latitude = 36.5999545, Longitude = -121.8851049
579: Latitude = 33.50139955, Longitude = -117.14875014393502
580: Latitude = 33.5171424,

...

20572: Latitude = 37.0356706, Longitude = -76.36192196664373
20573: Latitude = 38.4396675, Longitude = -78.87013652524797
20574: Latitude = 38.435104036911596, Longitude = -78.8464072142078
20575: Latitude = 38.406625112244896, Longitude = -78.90710934183674
20576: Latitude = 38.40663386734694, Longitude = -78.90711500510204
20577: Latitude = 38.40326379009431, Longitude = -78.91048938746992
20578: Latitude = 36.7626282, Longitude = -80.7347947
20579: Latitude = 37.7923282, Longitude = -79.4268677
20580: Latitude = 37.80253570266209, Longitude = -79.41076328640968
20581: Latitude = 37.802203, Longitude = -79.411506
20582: Latitude = 37.79388, Longitude = -79.880677
20583: Latitude = 38.677101224489796, Longitude = -78.42507153061224
20584: Latitude = 37.36998359169443, Longitude = -79.18190565501445
20585: Latitude = 37.369685, Longitude = -79.180381
20586: Latitude = 37.367805913156936, Longitude = -79.17523315726802
20587: Latitude = 37.36733215077553, Longitude = -79.18151758041598


20708: Latitude = 47.7028894, Longitude = -122.1781538
20709: Latitude = 47.64271655, Longitude = -122.19915060358818
20710: Latitude = 47.061959, Longitude = -122.7672008
20711: Latitude = 47.0367448, Longitude = -122.8076236
20712: Latitude = 47.04831208163265, Longitude = -122.82306157142857
20713: Latitude = 47.171402, Longitude = -122.483587
20714: Latitude = 47.6059602, Longitude = -120.6431138
20715: Latitude = 47.8168487, Longitude = -122.2910178
20716: Latitude = 47.8278417, Longitude = -122.3092665
20717: Latitude = 47.1301417, Longitude = -119.2780771
20718: Latitude = 47.1301417, Longitude = -119.2780771
20719: Latitude = 48.4385956, Longitude = -122.33589576862568
20720: Latitude = 47.01604538420527, Longitude = -124.16391913362182
20721: Latitude = 47.0495649, Longitude = -122.82411112208209
20722: Latitude = 48.1074165, Longitude = -123.4026641
20723: Latitude = 48.107931, Longitude = -123.4064334
20724: Latitude = 46.2734239, Longitude = -119.27583698012313
20725: Latit

20846: Latitude = 44.84296111713915, Longitude = -91.59479067697468
20847: Latitude = 44.843987, Longitude = -91.596864
20848: Latitude = 44.7875827, Longitude = -91.51819407380695
20849: Latitude = 42.65224523413623, Longitude = -88.54087859006032
20850: Latitude = 43.020352913793104, Longitude = -89.42200518965518
20851: Latitude = 43.0165949, Longitude = -89.42345110138928
20852: Latitude = 43.753813543164014, Longitude = -88.45266053056605
20853: Latitude = 43.7396285, Longitude = -88.449914
20854: Latitude = 43.76021385, Longitude = -88.47357727202069
20855: Latitude = 43.75446372676849, Longitude = -88.45947214703276
20856: Latitude = 43.21538781398447, Longitude = -88.11421719697992
20857: Latitude = 43.11557648346442, Longitude = -87.9167981925036
20858: Latitude = 43.1352881, Longitude = -87.9356423
20859: Latitude = 44.47257123347494, Longitude = -87.9554898756324
20860: Latitude = 44.5453423, Longitude = -88.0436695
20861: Latitude = 44.504264, Longitude = -88.055048
20862: 

20978: Latitude = 44.896142049999995, Longitude = -89.57153565251414
20979: Latitude = 44.904236, Longitude = -89.54955656041332
20980: Latitude = 42.83463569114742, Longitude = -88.75909675530526
20981: Latitude = 42.83176480905045, Longitude = -88.71143286950503
20982: Latitude = 43.2242567, Longitude = -89.3251011
20983: Latitude = 43.195935726668246, Longitude = -89.34395482229056
20984: Latitude = 43.6403997, Longitude = -89.763495
20985: Latitude = 43.6612094, Longitude = -89.7753325
20986: Latitude = 43.6266003, Longitude = -89.7775416
20987: Latitude = 43.6266003, Longitude = -89.7775416
20988: Latitude = 43.6266003, Longitude = -89.7775416
20989: Latitude = 43.6266003, Longitude = -89.7775416
20990: Latitude = 43.6266003, Longitude = -89.7775416
20991: Latitude = 43.6266003, Longitude = -89.7775416
20992: Latitude = 44.3539991, Longitude = -106.6937234
20993: Latitude = 42.85663535061851, Longitude = -106.32440213720314
20994: Latitude = 42.858449, Longitude = -106.332888
2099

21115: Latitude = 42.4157304, Longitude = -90.4294726
21116: Latitude = 41.8500302, Longitude = -87.9645077
21117: Latitude = 39.800764, Longitude = -89.654692
21118: Latitude = 41.73470985714286, Longitude = -85.97448714285714
21119: Latitude = 38.7834227, Longitude = -77.0146672
21120: Latitude = 39.0877782, Longitude = -76.7059047
21121: Latitude = 42.5560196, Longitude = -73.28917359472533
21122: Latitude = 45.373343, Longitude = -84.95533
21123: Latitude = 34.963578, Longitude = -90.009225
21124: Latitude = 34.9617605, Longitude = -89.8295315
21125: Latitude = 36.62798748019526, Longitude = -93.2773773727618
21126: Latitude = 36.6411357, Longitude = -93.2175285
21127: Latitude = 36.60670919219728, Longitude = -93.28811686198814
21128: Latitude = 36.6241315, Longitude = -93.26011
21129: Latitude = 38.1259916, Longitude = -92.6723254
21130: Latitude = 44.6549414, Longitude = -111.1013012
21131: Latitude = 36.1672559, Longitude = -115.1485163
21132: Latitude = 36.1672559, Longitude =

In [17]:
testdata.to_csv('./Data/data_with_geocodes.csv')